In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


# определим кодировки
#from chardet.universaldetector import UniversalDetector # Импортируем субмодуль chardet.universaldetector
#detector = UniversalDetector()
#with open('data/Справочник Номенклатуры.xlsx', 'rb') as fh:
#    for line in fh:
#        detector.feed(line)
#        if detector.done:
#            break
#detector.close()

nomen_data = pd.read_excel('data/Справочник Номенклатуры.xlsx')
sales_data = pd.read_excel('data/Продажи_дляП1.xlsx')
price_data = pd.read_excel('data/ПрайсДляП.xlsx')
stock_data = pd.read_excel('data/Остатки_дляП1.xlsx')

In [122]:

#Обработаем остатки
stock_data['Дата'] = pd.to_datetime(stock_data['ДАТА'], dayfirst=True) 
stock_data['Артикул'] = stock_data['АРТИКУЛ']
stock_data = stock_data.drop(['ДАТА','Маркер', 'АРТИКУЛ'],axis=1)

In [123]:
sales_data['Дата'] = pd.to_datetime(sales_data['Дата'], dayfirst=True)
sales_data['Дата'] = sales_data['Дата'].dt.date
sales_data['Дата'] = pd.to_datetime(sales_data['Дата'], dayfirst=True)
sales_data['Артикул'] = sales_data['Артикул']

In [124]:
display(nomen_data.head())
display(sales_data.head())
display(price_data.head())
display(stock_data.head())

,Артикул,Номенклатура,Вес,Объем,Коэффициент,Номенклатурная группа,Ассортиментная группа,Проект,Категория,Основной поставщик,Артикул производителя,Статус
0,182260/п,"Бокал д/вина, 0.2л, кристалл, ПС",9.00,0.060,79.0,Бокалы,Бокалы,1. Одноразовая посуда,X5 Retail Group,АССОРТИ-ТВЕРЬ ООО,NaN,Действующая
1,182261,"Бокал д/вина, 0.20л, кристалл, ПС",1.20,0.021,12.0,Бокалы,Бокалы,1. Одноразовая посуда,X5 Retail Group,АССОРТИ-ТВЕРЬ ООО,182261,Действующая
2,182272//,"Бокал д/мартини, 0.1л, прозрачн., кристалл, ПС",1.96,0.064,16.0,Бокалы,Бокалы,1. Одноразовая посуда,АШАН,ВЗЛП ООО,1008,Действующая
3,182271/в,"Бокал д/вина, 0.2л, прозрачн., кристалл, ПС",3.50,0.025,10.0,Бокалы,Бокалы,1. Одноразовая посуда,Маркетплейс Wildberries,ВЗЛП ООО,1011,Действующая
4,182272/в,"Бокал д/мартини, 0.1л, прозрачн., кристалл, ПС",3.20,0.075,32.0,Бокалы,Бокалы,1. Одноразовая посуда,Маркетплейс Wildberries,ВЗЛП ООО,1008,Действующая


,Артикул,КЛИЕНТ,Менеджер,Дата,Продажи бе
0,440038,ВОСТОК-ЗАПАД ООО,Поспелова Елена Олеговна,2022-06-06,4900.0
1,440038а,ВОСТОК-ЗАПАД ООО,Поспелова Елена Олеговна,2022-06-06,4000.0
2,153501ч,ИНТЕРНЕТ РЕШЕНИЯ ООО,Кузнецова Татьяна Анатольевна,2022-06-06,2.0
3,159501,ИНТЕРНЕТ РЕШЕНИЯ ООО,Кузнецова Татьяна Анатольевна,2022-06-06,2.0
4,153501ч,ИНТЕРНЕТ РЕШЕНИЯ ООО,Кузнецова Татьяна Анатольевна,2022-06-06,2.0


,Артикул,Цена
0,461620гр,42.00
1,481610,171.85
2,477083б,965.19
3,47630200,207.05
4,268348а/,121.48


,Остаток бе,Дата,Артикул
0,626.4,2022-06-06,100130
1,188.0,2022-06-06,100160
2,31.5,2022-06-06,103301
3,28.0,2022-06-06,103321
4,7.0,2022-06-06,103331


In [125]:
myst_data = pd.merge(left = stock_data, right= sales_data, on = ['Дата','Артикул'], how = 'outer')
myst_data = pd.merge(left = myst_data, right= price_data, on = 'Артикул', how = 'left')
myst_data = pd.merge(left = myst_data, right= nomen_data, on = 'Артикул', how = 'left')
myst_data['Продажи руб'] = myst_data['Продажи бе'] * myst_data['Цена']
myst_data['Остаток руб'] = myst_data['Остаток бе'] * myst_data['Цена']
display(myst_data.head())

,Остаток бе,Дата,Артикул,КЛИЕНТ,Менеджер,Продажи бе,Цена,Номенклатура,Вес,Объем,Коэффициент,Номенклатурная группа,Ассортиментная группа,Проект,Категория,Основной поставщик,Артикул производителя,Статус,Продажи руб,Остаток руб
0,626.4,2022-06-06,100130,NaN,NaN,NaN,661.50,"Стакан пищев., д/хол./гор., 0.1л, п/прозрачн., ПП",8.0,0.084,2.7,Стаканы ПП до 200 мл прозрачные,Стаканы,1. Одноразовая посуда,под заказ,ПРОИЗВОДСТВЕННАЯ КОМПАНИЯ АЛИТ ПЛАСТ ООО,6532,Действующая,NaN,414363.60
1,188.0,2022-06-06,100160,NaN,NaN,NaN,391.23,"Стакан пищев., д/хол./гор., 0.1л, п/прозрачн., ПП",6.0,0.062,4.0,Стаканы ПП до 200 мл прозрачные,Стаканы,1. Одноразовая посуда,NaN,ПолиЭр ООО,-,Действующая,NaN,73551.24
2,31.5,2022-06-06,103301,NaN,NaN,NaN,723.10,"Стакан хол/гор, 0.20л, зелён., ПП",10.6,0.105,3.5,Стаканы ПП 200 мл цветные,Стаканы,1. Одноразовая посуда,под заказ,ПЛАСТ ИНДУСТРИЯ ООО,NaN,Действующая,NaN,22777.65
3,28.0,2022-06-06,103321,NaN,NaN,NaN,723.10,"Стакан хол/гор, 0.20л, красн., ПП",10.6,0.105,3.5,Стаканы ПП 200 мл цветные,Стаканы,1. Одноразовая посуда,под заказ,ПЛАСТ ИНДУСТРИЯ ООО,NaN,Действующая,NaN,20246.80
4,7.0,2022-06-06,103331,NaN,NaN,NaN,723.10,"Стакан хол/гор, 0.20л, желт., ПП",9.1,0.105,3.5,Стаканы ПП 200 мл цветные,Стаканы,1. Одноразовая посуда,под заказ,ПЛАСТ ИНДУСТРИЯ ООО,NaN,Действующая,NaN,5061.70


In [135]:
df = myst_data[myst_data['Артикул'] == 120100]
df_sales = df.groupby(
                                by=['Дата'],
                                as_index=False
                                )['Продажи руб'].sum()
df_stock = df.groupby(
                                by=['Дата'],
                                as_index=False
                                )['Остаток руб'].mean()

df = pd.merge(left = df_stock, right= df_sales, on = 'Дата', how = 'outer')
df = df.fillna({'Продажи руб':0, 'Остаток руб':0})

df['медиана'] = df['Продажи руб'].median()
df['среднее'] = df['Продажи руб'].mean()
df['std'] = df['Продажи руб'].std()
df['страховой запас на день'] = df['Продажи руб'].mean() * ( 1+ df['std']/df['Продажи руб'].mean())
df['страховой запас'] = df['страховой запас на день']*5
display(df.head())
fig = px.line(df, x="Дата", y=['Продажи руб','Остаток руб', 'медиана','среднее','страховой запас на день','страховой запас'] ) 
fig.show()

fig = px.box(df, x='Продажи руб')
fig.show()

,Дата,Остаток руб,Продажи руб,медиана,среднее,std,страховой запас на день,страховой запас
0,2022-06-06,1053913.128,15301.824,33472.74,40406.379,33793.042273,74199.421273,370997.106363
1,2022-06-07,985054.920,65032.752,33472.74,40406.379,33793.042273,74199.421273,370997.106363
2,2022-06-08,956364.000,28690.920,33472.74,40406.379,33793.042273,74199.421273,370997.106363
3,2022-06-09,942974.904,13389.096,33472.74,40406.379,33793.042273,74199.421273,370997.106363
4,2022-06-10,914283.984,28690.920,33472.74,40406.379,33793.042273,74199.421273,370997.106363
